In [1]:
# define Spark client

from pyspark.sql import SparkSession


spark = SparkSession.builder \
    .config("spark.sql.warehouse.dir", "/usr/local/hadoop/warehouse") \
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg.type", "hive") \
    .config("spark.sql.catalog.iceberg.uri", "thrift://hivemetastore:9083") \
    .config("spark.sql.catalog.iceberg.cache-enabled", False) \
    .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/12 21:12:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/12 21:12:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/03/12 21:12:46 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
# query existing snapshots

spark.sql("""
SELECT snapshot_id, committed_at FROM iceberg.default.bank_transfers.snapshots
ORDER BY committed_at           
""").show(truncate=False)

+-------------------+-----------------------+
|snapshot_id        |committed_at           |
+-------------------+-----------------------+
|7385819794904720348|2023-03-12 21:04:49.206|
|7207270143656420745|2023-03-12 21:04:53.598|
|4886137892140171718|2023-03-12 21:05:02.248|
+-------------------+-----------------------+



In [9]:
# query a snapshot using its id

spark.sql("""
SELECT * FROM iceberg.default.bank_transfers VERSION AS OF 10963874102873;
""").show(truncate=False)

ParseException: 
mismatched input 'AS' expecting {<EOF>, ';'}(line 2, pos 53)

== SQL ==

SELECT * FROM iceberg.default.bank_transfers VERSION AS OF 10963874102873;
-----------------------------------------------------^^^


In [ ]:
TODO:
    - expand schema
    - change partitioning scheme and insert more data
    - delete column and recreate it to show it won't "bring back" old data
    - perform time travel and show a self join or something to show what was inserted or removed